### loading in the GEM-PRO dataframe

In [1]:
import pandas as pd

In [10]:
GP = pd.read_csv('/home/nathan/Dropbox/Shared Folders/GEMPRO/' + 'DF_GEMPRO.csv', index_col=0)
# forcing gene IDs to be read as strings
GP['m_gene_original'] = GP['m_gene_original'].astype(str)
GP['m_gene_entrez'] = GP['m_gene_entrez'].astype(str)
GP['m_gene_isoform'] = GP['m_gene_isoform'].astype(str)
GP.head()

,m_gene_original,m_gene_entrez,m_gene_isoform,u_uniprot_acc,u_isoform_id,u_refseq,u_ensp,u_seq_len,u_seq,u_reviewed,...,ssb_p_aln_coverage,ssb_p_percent_seq_ident,ssb_p_no_deletions_in_pdb,ssb_p_aln_coverage_sim,ssb_si_score,ssb_rez_score,ssb_raw_score,ssb_above_cutoffs,ssb_rank,ssb_best_file
0,100.1,100,1,P00813,P00813-1,NP_000013,ENSP00000361965,363,MAQTPAFDKPKVELHVHLDGSIKPETILYYGRRRGIALPANTAEGL...,True,...,359,0.988981,True,359,1.586721,1.213333,2.800054,True,1,3iar.pdb
1,10005.1,10005,1,O14734,O14734-1,NP_005460,ENSP00000217455,319,MSSPQAPEDGQGCGDRGDPPGDLRSVLVTTVLNLEPLDEDLFRGRH...,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NP_005460.2_model1_fix.pdb
2,10005.2,10005,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10005.3,10005,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10007.1,10007,1,P46926,P46926-1,NP_005462,ENSP00000311876,289,MKLIILEHYSQASEWAAKYIRNRIIQFNPGPEKYFTLGLPTGSTPL...,True,...,281,0.972318,True,281,1.559988,1.175000,2.734988,True,1,1ne7.pdb


### writing FASTA files

In [34]:
import os.path
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC

def write_fasta_file(sequence, fileout):
    '''
    This writes a fasta file for a SeqRecord object. It also checks if the file exists already and returns the filename.
    
    Input: sequence - Biopython SeqRecord object, identification - ID of the sequence.
    Output: Filename of fasta file
    '''
    
    outfile = "%s" % fileout
    if os.path.isfile(outfile):
        print 'FASTA file already exists %s' % outfile
        return outfile
    else:
        SeqIO.write(sequence, outfile, "fasta")
        return outfile

In [52]:
# example: for gene 100.1

# getting the IDs and making output file name
seq_id = GP[GP.m_gene_original == '100.1'].u_isoform_id.values[0]
seq_output = '/tmp/' + seq_id + '.faa'

# getting the sequence and making it into a Biopython SeqRecord object
seq = GP[GP.m_gene_original == '100.1'].u_seq.values[0]
seq_biop = SeqRecord(Seq(seq, IUPAC.protein),id=seq_id,description='uniprot sequence')

# writing the SeqRecord object (formats it as FASTA file)
out_file = write_fasta_file(seq_biop, seq_output)

FASTA file already exists /tmp/P00813-1.faa


In [53]:
# just saving in tmp for this example, all fasta files were already written in "sequence_files"
!cat $out_file

>P00813-1 uniprot sequence
MAQTPAFDKPKVELHVHLDGSIKPETILYYGRRRGIALPANTAEGLLNVIGMDKPLTLPD
FLAKFDYYMPAIAGCREAIKRIAYEFVEMKAKEGVVYVEVRYSPHLLANSKVEPIPWNQA
EGDLTPDEVVALVGQGLQEGERDFGVKARSILCCMRHQPNWSPKVVELCKKYQQQTVVAI
DLAGDETIPGSSLLPGHVQAYQEAVKSGIHRTVHAGEVGSAEVVKEAVDILKTERLGHGY
HTLEDQALYNRLRQENMHFEICPWSSYLTGAWKPDTEHAVIRLKNDQANYSLNTDDPLIF
KSTLDTDYQMTKRDMGFTEEEFKRLNINAAKSSFLPEDEKRELLDLLYKAYGMPPSASAG
QNL


### aligning two FASTA files

all sequences in the dataframe (uniprot and PDB) are already saved as separate FASTA files, so we can use these for the EMBOSS needle alignment tool

**for this function to run, the EMBOSS needle tool needs to run correctly from your command line**

- on ubuntu you can try just installing it like this: 
    
    
    > sudo apt-get install emboss
    
- and then to test it running it from the command line, it should ask you for an input sequence/output sequence:
    
    
    > needle

In [45]:
import os.path
from Bio.Emboss.Applications import NeedleCommandline

def run_alignment(fasta1_id, fasta1, fasta2_id, fasta2):
    '''
    Runs the needle alignment program and writes the result to a file. Returns the filename. Standard gap inputs are used.
    
    Input:  fasta1 - fasta file name ("reference" sequence)
            fasta2 - fasta file name (what you're interested in aligning)
    Output: alignment_file - file name of alignment
    '''

    alignment_file = "%s_%s_align.txt" % (fasta1_id, fasta2_id)
    
    if os.path.isfile(alignment_file):
        print 'Alignment %s file already exists' % alignment_file
        return alignment_file

    else:
        print '**RUNNING ALIGNMENT FOR %s AND %s**' % (fasta1_id, fasta2_id)
        needle_cline = NeedleCommandline(asequence=fasta1, bsequence=fasta2, gapopen=10, gapextend=0.5, outfile=alignment_file)
        stdout, stderr = needle_cline()
        return alignment_file

In [69]:
# example: aligning 100.1 to 3iar.pdb

SEQUENCE_FILES = '/home/nathan/Dropbox/Shared Folders/GEMPRO/sequence_files/'
UNIPROT_FILES = os.path.join(SEQUENCE_FILES, 'uniprot_sequences')
PDB_SEQ_FILES = os.path.join(SEQUENCE_FILES, 'pdb_sequences')

# 1. get the uniprot sequence file
seq_id = GP[GP.m_gene_original == '100.1'].u_isoform_id.values[0]
seq_fasta = os.path.join(UNIPROT_FILES, seq_id + '.faa')

if os.path.exists(seq_fasta):
    print('found uniprot fasta file {}'.format(seq_fasta))
    
# 2. get the pdb sequence file
pdb_id = GP[GP.m_gene_original == '100.1'].ssb_best_file.values[0].strip('.pdb')
pdb_fasta = os.path.join(PDB_SEQ_FILES, pdb_id + '.faa')

if os.path.exists(pdb_fasta):
    print('found pdb fasta file {}'.format(pdb_fasta))
    
# 3. run the alignment using the function above
os.chdir('/tmp/')
alignment_filename = run_alignment(seq_id, seq_fasta, pdb_id, pdb_fasta)

found uniprot fasta file /home/nathan/Dropbox/Shared Folders/GEMPRO/sequence_files/uniprot_sequences/P00813-1.faa
found pdb fasta file /home/nathan/Dropbox/Shared Folders/GEMPRO/sequence_files/pdb_sequences/3iar.faa
**RUNNING ALIGNMENT FOR P00813-1 AND 3iar**


In [72]:
!cat $alignment_filename

########################################
# Program: needle
# Rundate: Mon  1 Feb 2016 15:13:13
# Commandline: needle
#    -outfile P00813-1_3iar_align.txt
#    -asequence "/home/nathan/Dropbox/Shared Folders/GEMPRO/sequence_files/uniprot_sequences/P00813-1.faa"
#    -bsequence "/home/nathan/Dropbox/Shared Folders/GEMPRO/sequence_files/pdb_sequences/3iar.faa"
#    -gapopen 10
#    -gapextend 0.5
# Align_format: srspair
# Report_file: P00813-1_3iar_align.txt
########################################

#=======================================
#
# Aligned_sequences: 2
# 1: P00813-1
# 2: 3iar.A
# Matrix: EBLOSUM62
# Gap_penalty: 10.0
# Extend_penalty: 0.5
#
# Length: 364
# Identity:     359/364 (98.6%)
# Similarity:   359/364 (98.6%)
# Gaps:           5/364 ( 1.4%)
# Score: 1889.0
# 
#
#=======================================

P00813-1           1 MAQTPAFDKPKVELHVHLDGSIKPETILYYGRRRGIALPANTAEGLLNVI     50
                         ||||||||||||||||||||||||||||||||||||||||||||||
3iar.A           

### loading alignment file as a dataframe

- this code parses a needle alignment file and makes it into a dataframe
    - each row of the dataframe is a position in the reference sequence
    - it also tells you what parts are deleted, mutated, etc.

In [80]:
import numpy as np
from Bio import AlignIO
from collections import defaultdict

def get_alignment_allpos_df(alignment_file, a_seq_id=None, b_seq_id=None):
    '''
    Takes in a needle alignment file and returns a pandas dataframe of the results
    Input: alignment_file - the path to the alignment file, 
            a_seq_id - optional ID of the reference sequence, 
            b_seq_id - optional ID of the second sequence
    Output: alignment_df - a pandas dataframe of the alignment results
    '''
    alignments = list(AlignIO.parse(alignment_file, "emboss"))

    appender = defaultdict(dict)
    idx = 0
    for alignment in alignments:
    #         if not switch:
        if not a_seq_id:
            a_seq_id = list(alignment)[0].id
        a_seq = str(list(alignment)[0].seq)
        if not b_seq_id:
            b_seq_id = list(alignment)[1].id
        b_seq = str(list(alignment)[1].seq)

        a_idx = 1
        b_idx = 1

        for i, (a,b) in enumerate(zip(a_seq,b_seq)):
            if a == b and a != '-' and b != '-':
                aa_flag = 'match'
            if a != b and a == '-' and b != '-':
                aa_flag = 'insertion'
            if a != b and a != '-' and b == '-':
                aa_flag = 'deletion'
            if a != b and a != '-' and b == 'X':
                aa_flag = 'unresolved'
            if a != b and b != '-' and a == 'X':
                aa_flag = 'unresolved'
            elif a != b and a != '-' and b != '-':
                aa_flag = 'mutation'
                
            appender[idx]['id_a'] = a_seq_id
            appender[idx]['id_b'] = b_seq_id
            appender[idx]['type'] = aa_flag
            
            if aa_flag == 'match' or aa_flag == 'unresolved' or aa_flag == 'mutation':
                appender[idx]['id_a_aa'] = a
                appender[idx]['id_a_pos'] = a_idx
                appender[idx]['id_b_aa'] = b
                appender[idx]['id_b_pos'] = b_idx
                a_idx += 1
                b_idx += 1

            if aa_flag == 'deletion':
                appender[idx]['id_a_aa'] = a
                appender[idx]['id_a_pos'] = a_idx
                a_idx += 1

            if aa_flag == 'insertion':
                appender[idx]['id_b_aa'] = b
                appender[idx]['id_b_pos'] = b_idx
                b_idx += 1
            
            idx += 1

    alignment_df = pd.DataFrame.from_dict(appender, orient='index')
    alignment_df = alignment_df[['id_a', 'id_b', 'type', 'id_a_aa', 'id_a_pos', 'id_b_aa', 'id_b_pos']].fillna(value=np.nan)
    
    return alignment_df

In [81]:
get_alignment_allpos_df(alignment_filename)

,id_a,id_b,type,id_a_aa,id_a_pos,id_b_aa,id_b_pos
0,P00813-1,3iar.A,deletion,M,1,NaN,NaN
1,P00813-1,3iar.A,deletion,A,2,NaN,NaN
2,P00813-1,3iar.A,deletion,Q,3,NaN,NaN
3,P00813-1,3iar.A,deletion,T,4,NaN,NaN
4,P00813-1,3iar.A,match,P,5,P,1
5,P00813-1,3iar.A,match,A,6,A,2
6,P00813-1,3iar.A,match,F,7,F,3
7,P00813-1,3iar.A,match,D,8,D,4
8,P00813-1,3iar.A,match,K,9,K,5
9,P00813-1,3iar.A,match,P,10,P,6
